In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from radial import orbit, estimate, dataset
%matplotlib inline

In [ ]:
w16 = dataset.RVDataSet('../tests/HIP67620_WF16.dat', t_offset=-5E4, rv_offset='subtract_mean', 
                         instrument_name='W16', target_name='HIP 67620', t_col=1, 
                         rv_col=3, rv_unc_col=4)
w16.plot()

In [ ]:
guess = {'log_k': np.log10(6314),
         'log_period': np.log10(3819.2),
         't0': 4904.5,
         'omega': 139.3 * np.pi / 180,
         'log_ecc': np.log10(0.343),
         'gamma_0': 401,
         'gamma_1': 3629,
         'gamma_2': 127,
         'sigma_0': 3,
         'sigma_1': 299,
         'sigma_2': 154}

bounds = {'log_k': (3, 4),
          'log_period': (3.0, 4.0),
          't0': (4000, 6000),
          'omega': (120 * np.pi / 180, 160 * np.pi / 180),
          'log_ecc': (-0.5, -0.4),
          'gamma_0': (0, 1000),
          'gamma_1': (3000, 4000),
          'gamma_2': (0, 1000),
          'sigma_0': (0, 1000),
          'sigma_1': (0, 1000),
          'sigma_2': (0, 1000)}

estim = estimate.FullOrbit([w16], guess, bounds, use_add_sigma=False, 
                           parametrization='mc10')
plot = estim.plot_ds(plot_guess=True, fold=False, legend_loc=2)
plt.show()

In [ ]:
result = estim.lmfit_orbit(update_guess=True)

In [ ]:
ax, fig = estim.plot_ds(plot_guess=True, fold=False, legend_loc=4)